### 先清洗一波我的训练数据

In [1]:
### 27号我有一波新的训练数据，看看成色。
import os, json
import re

In [2]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/papers/hidden_cot/data/GSM8K/hcot_gsm8k.jsonl",
    "r",
) as f:
    hcot_gsm8k_new = f.readlines()

In [3]:
def fix_incomplete_tags_v2(text: str) -> str:
    tags = ["thought", "equation"]

    for tag in tags:
        open_tag = f"<{tag}>"
        close_tag = f"</{tag}>"
        incomplete_situation_open_tag_left = f"<{tag}"
        incomplete_situation_open_tag_right = f"{tag}>"
        incomplete_situation_close_tag_left = f"/{tag}>"
        incomplete_situation_close_tag_right = f"</{tag}"
        text_tmp = text.replace(open_tag, "&*#$%")
        text_tmp = text_tmp.replace(close_tag, "@#!$%")
        text_tmp = text_tmp.replace(incomplete_situation_close_tag_left, close_tag)
        text_tmp = text_tmp.replace(close_tag, "@#!$%")

        text_tmp = text_tmp.replace(incomplete_situation_close_tag_right, close_tag)
        text_tmp = text_tmp.replace(close_tag, "@#!$%")

        text_tmp = text_tmp.replace(incomplete_situation_open_tag_left, open_tag)
        text_tmp = text_tmp.replace(open_tag, "&*#$%")
        text_tmp = text_tmp.replace(incomplete_situation_open_tag_right, open_tag)
        text_tmp = text_tmp.replace(open_tag, "&*#$%")

        text_tmp = text_tmp.replace("&*#$%", open_tag)
        text_tmp = text_tmp.replace("@#!$%", close_tag)
        text = text_tmp
    return text


# Example usage
example_string = "This is a test equation>xample without proper tag. Here is another \n/thought> with no end."
fixed_string = fix_incomplete_tags_v2(example_string)
fixed_string

'This is a test <equation>xample without proper tag. Here is another \n</thought> with no end.'

In [4]:
json.loads(hcot_gsm8k_new[0])

{'question': 'USER: Question: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\n Analysis: Tony got twice $1750 which is 2*$1750 = $<<2*1750=3500>>3500\nThe total amount shared was $1750+$3500 = $<<1750+3500=5250>>5250\n Answer:5250',
 'raw_result': {'id': 'chatcmpl-8SRAP0aNifsGKlamyFXbqSkeQiSKe',
  'object': 'chat.completion',
  'created': 1701788305,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how much money Mr. Sam sha

In [5]:
new_hcot_gsm8k_new = []
for line in hcot_gsm8k_new:
    line_js = json.loads(line)
    if line_js["response"]:
        response_origin = line_js["response"]
        response_fix = fix_incomplete_tags_v2(response_origin)
        line_js["response"] = response_fix
        new_hcot_gsm8k_new.append(line_js)

In [6]:
with open("./data/GSM8K/hcot_gsm8k_fix.jsonl", "w") as f:
    for line in new_hcot_gsm8k_new:
        f.write(json.dumps(line, ensure_ascii=False)+"\n")

### 构建带cot的直接训练数据

In [7]:
import os, json
from tqdm import tqdm

with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/hcot_gsm8k_fix.jsonl",
    "r",
) as f:
    h_cot_gsm8k = f.readlines()


def build_direct_conversations(one_gsm8k_input):
    one_gsm8k_input["response"] = one_gsm8k_input["response"].replace(
        "[assistant'](#math_action)", "[assistant](#math_action)"
    )
    model_response = (
        one_gsm8k_input["response"]
        .split("[assistant](#math_action)")[1]
        .strip("\\n")
        .replace("\\n", "\n")
    )
    one_gsm8k_input["conversations"] = [
        {"from": "human", "value": one_gsm8k_input["meta"]["question"]},
        {"from": "gpt", "value": model_response},
    ]
    return one_gsm8k_input


h_cot_gsm8k_direct = []
for one_gsm8k in h_cot_gsm8k:
    one_gsm8k_js = json.loads(one_gsm8k)
    if one_gsm8k_js["response"]:
        h_cot_gsm8k_direct.append(build_direct_conversations(one_gsm8k_js))

In [8]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/h_cot_direct_conv.json",
    "w",
) as f:
    json.dump(h_cot_gsm8k_direct, f, ensure_ascii=False)

In [9]:
len(h_cot_gsm8k_direct)

7430

In [10]:
h_cot_gsm8k_direct[0]

{'question': 'USER: Question: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\n Analysis: Tony got twice $1750 which is 2*$1750 = $<<2*1750=3500>>3500\nThe total amount shared was $1750+$3500 = $<<1750+3500=5250>>5250\n Answer:5250',
 'raw_result': {'id': 'chatcmpl-8SRAP0aNifsGKlamyFXbqSkeQiSKe',
  'object': 'chat.completion',
  'created': 1701788305,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how much money Mr. Sam sha

#### 构建hcot的cot model的训练数据

In [11]:
import re


def remove_thoughts(text):
    """
    Removes all occurrences of <thought>xxx</thought> from the text.

    Args:
    text (str): The input string containing <thought>xxx</thought> tags.

    Returns:
    str: The modified string with <thought>xxx</thought> tags removed.
    """
    # Pattern to match <thought>xxx</thought> and optional preceding and following newlines
    pattern = r"(?<=\n)<thought>.*?</thought>(?=\n)"

    # Replace all occurrences with an empty string
    modified_text = re.sub(pattern, "<[COT]>", text, flags=re.DOTALL)

    return modified_text


def extract_thoughts_accumulative(problem, text):
    pairs = []
    previous_end = 0
    # prefix = f"Solve the following math word problem with your step-by-step analysis, {problem}. I will provide the previous analysis and equations, your job is to show me your thought of the next action, you do not need to give the calculation equation.\n\n"
    prefix = f"{problem}"

    while "<thought>" in text[previous_end:]:
        # Find the next <thought> tag
        start_thought = text.find("<thought>", previous_end)
        end_thought = text.find("</thought>", start_thought)

        # Break the loop if no more <thought> tag is found
        if start_thought == -1 or end_thought == -1:
            break

        # Extract the input and output
        input_text = text[:start_thought].strip()
        output_text = text[
            start_thought + 9 : end_thought
        ].strip()  # 9 is the length of '<thought>'

        # Append the pair to the result
        input_text_tmp = prefix + remove_thoughts(input_text)

        pairs.append(
            {
                "conversations": [
                    {"from": "human", "value": input_text_tmp + "\n<[COT]>"},
                    {"from": "gpt", "value": output_text},
                ],
                "Query": input_text_tmp,
                "COT_target": "\n<[COT]>",
                "thought_target": output_text,
            }
        )

        # Update the end position for the next iteration
        previous_end = end_thought + 10

    return pairs

In [12]:
cot_model_train_data = []
for one_gsm8k_direct in h_cot_gsm8k_direct:
    new_train_data = extract_thoughts_accumulative(
        one_gsm8k_direct["conversations"][0]["value"],
        one_gsm8k_direct["conversations"][1]["value"],
    )
    cot_model_train_data.extend(new_train_data)

In [64]:
cot_model_train_data = []
for one_gsm8k_direct in h_cot_gsm8k_direct:
    new_train_data = extract_thoughts_accumulative(
        one_gsm8k_direct["conversations"][0]["value"],
        one_gsm8k_direct["conversations"][1]["value"],
    )
    cot_model_train_data.extend(new_train_data)

In [13]:
cot_model_train_data[0]

{'conversations': [{'from': 'human',
   'value': 'Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.\n<[COT]>'},
  {'from': 'gpt',
   'value': "Firstly, since Tony got twice as much money as Ken did, we can find out how much Tony got by multiplying Ken's share by 2."}],
 'Query': 'Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.',
 'COT_target': '\n<[COT]>',
 'thought_target': "Firstly, since Tony got twice as much money as Ken did, we can fi

In [14]:
h_cot_gsm8k_direct[0]["conversations"][1]["value"]

"The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.\n<thought>Firstly, since Tony got twice as much money as Ken did, we can find out how much Tony got by multiplying Ken's share by 2.</thought>\n<equation>Tony's share = 2 * $1750 = $3500</equation>\n<thought>Next, to find out the total amount of money shared by Mr. Sam between his two sons, we add up what each son received.</thought>\n<equation>Total amount shared = $1750 (Ken's share) + $3500 (Tony's share) = $5250</equation>\n\nSo, Mr. Sam shared a total of $5250 between his two sons."

In [66]:
jesus = extract_thoughts_accumulative(
    h_cot_gsm8k_direct[0]["conversations"][0]["value"],
    h_cot_gsm8k_direct[0]["conversations"][1]["value"],
)

In [15]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/COT_model_train_data_kun_idea.json",
    "w",
) as f:
    json.dump(cot_model_train_data, f, ensure_ascii=False)

#### 构建hcot-model-llm部分的训练数据

In [16]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/h_cot_direct_conv.json",
    "r",
) as f:
    h_cot_gsm8k_direct = json.load(f)

In [17]:
import re


def remove_thoughts(text):
    """
    Removes all occurrences of <thought>xxx</thought> from the text.

    Args:
    text (str): The input string containing <thought>xxx</thought> tags.

    Returns:
    str: The modified string with <thought>xxx</thought> tags removed.
    """
    # Pattern to match <thought>xxx</thought> and optional preceding and following newlines
    pattern = r"(?<=\n)<thought>.*?</thought>(?=\n)"

    # Replace all occurrences with an empty string
    modified_text = re.sub(pattern, "<[COT]>", text, flags=re.DOTALL)

    return modified_text

In [ ]:
one_text = "The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.\n<thought>Firstly, since Tony got twice as much money as Ken did, we can find out how much Tony got by multiplying Ken's share by 2.</thought>\n<equation>Tony's share = 2 * $1750 = $3500</equation>\n<thought>Next, to find out the total amount of money shared by Mr. Sam between his two sons, we add up what each son received.</thought>\n<equation>Total amount shared = $1750 (Ken's share) + $3500 (Tony's share) = $5250</equation>\n\nSo, Mr. Sam shared a total of $5250 between his two sons."
remove_thoughts(one_text)

"The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.\n<[COT]>\n<equation>Tony's share = 2 * $1750 = $3500</equation>\n<[COT]>\n<equation>Total amount shared = $1750 (Ken's share) + $3500 (Tony's share) = $5250</equation>\n\nSo, Mr. Sam shared a total of $5250 between his two sons."

In [18]:
h_cot_gsm8k_hcot_llm_model = []
for data in h_cot_gsm8k_direct:
    conv = data["conversations"]
    conv_response = conv[1]["value"]
    conv_response_with_cot = remove_thoughts(conv_response)
    conv[1]["value"] = conv_response_with_cot
    h_cot_gsm8k_hcot_llm_model.append(data)

In [19]:
h_cot_gsm8k_hcot_llm_model[0]

{'question': 'USER: Question: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\n Analysis: Tony got twice $1750 which is 2*$1750 = $<<2*1750=3500>>3500\nThe total amount shared was $1750+$3500 = $<<1750+3500=5250>>5250\n Answer:5250',
 'raw_result': {'id': 'chatcmpl-8SRAP0aNifsGKlamyFXbqSkeQiSKe',
  'object': 'chat.completion',
  'created': 1701788305,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how much money Mr. Sam sha

In [ ]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/hcot_llm_data.json",
    "w",
) as f:
    json.dump(h_cot_gsm8k_hcot_llm_model, f, ensure_ascii=False)

In [ ]:
import os,json

In [ ]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/papers/hidden_cot/data/GSM8K/hcot_llm_data.json", "r") as f:
    file = json.load(f)

In [ ]:
file[0]

{'question': 'USER: Question: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?\n Analysis: Tony got twice $1750 which is 2*$1750 = $<<2*1750=3500>>3500\nThe total amount shared was $1750+$3500 = $<<1750+3500=5250>>5250\n Answer:5250',
 'raw_result': {'id': 'chatcmpl-8SRAP0aNifsGKlamyFXbqSkeQiSKe',
  'object': 'chat.completion',
  'created': 1701788305,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how much money Mr. Sam sha

In [ ]:
file[1]

{'question': 'USER: Question: A car is driving through a tunnel with many turns. After a while, the car must travel through a ring that requires a total of 4 right-hand turns. After the 1st turn, it travels 5 meters. After the 2nd turn, it travels 8 meters. After the 3rd turn, it travels a little further and at the 4th turn, it immediately exits the tunnel. If the car has driven a total of 23 meters around the ring, how far did it have to travel after the 3rd turn?\n Analysis: From the details given, the car has traveled 5 meters at the 1st turn + 8 meters after the 2nd turn + 0 meters after the 4th turn = <<5+8+0=13>>13 meters around the ring.\nIt must therefore have driven 23 total meters – 13 calculated meters = 10 meters after the 3rd turn.\n Answer:10',
 'raw_result': {'id': 'chatcmpl-8SRAPK62ihju3TAq9F26E4ooQdZeq',
  'object': 'chat.completion',
  'created': 1701788305,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '<|im_start

In [ ]:
has_thought = []
for one in file:
    one_content = one["conversations"][1]["value"]
    if "<thought>" in one_content:
        has_thought.append(one)

In [ ]:
len(has_thought)

248

In [ ]:
has_thought[0]

{'question': 'USER: Question: A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all?\n Analysis: Mr. Benson had a 5% discount for each of the 12 - 10 = <<12-10=2>>2 tickets.\nSo, those two tickets had a $40 x 5/100 = $<<40*5/100=2>>2 discount each.\nHence, each ticket cost $40 - $2 = $<<40-2=38>>38 each.\nThus, two discounted tickets amount to $38 x 2 = $<<38*2=76>>76.\nAnd the other ten tickets amount to $40 x 10 = $<<40*10=400>>400.\nHence, Mr. Benson paid a total of $400 + $76 = $<<400+76=476>>476.\n Answer:476',
 'raw_result': {'id': 'chatcmpl-8SRB5JiwuLiCvgGboHoupKbLk8ywt',
  'object': 'chat.completion',
  'created': 1701788347,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my